<a href="https://colab.research.google.com/github/tanishkamaurya/Real-Time-Parking-Pricing/blob/main/Capstone_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh panel --quiet


In [ ]:
!pip install google-cloud-bigquery --upgrade

In [ ]:
!pip install pathway --upgrade

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
# from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()


In [ ]:
from google.colab import drive
file_id = '1RqHF3zphAFOtYZgReDJUxEFweOiVAxqP'
url = f'https://drive.google.com/uc?id={file_id}'

df = pd.read_csv(url)
df.head()

In [ ]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))


In [ ]:
# This cell is no longer needed as the data is loaded in cell yzdfFDdQssbL
# filename = "dataset.csv"
# df = pd.read_csv(f'https://drive.google.com/file/d/1RqHF3zphAFOtYZgReDJUxEFweOiVAxqP/view?usp=drive_link')
# df.to_csv(filename, index=False)

In [ ]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

df.rename(columns={'SystemCodeNumber': 'LocationID'}, inplace=True)

df[['Timestamp', 'LocationID', 'Occupancy', 'Capacity', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']].to_csv('parking_stream.csv', index=False)

In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    LocationID: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str # Changed to string based on the original dataframe
    IsSpecialDay: int # Changed to int based on the original dataframe
    VehicleType: str

is_special_day = pw.this.IsSpecialDay * 1.0


In [ ]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


MODEL 1 : BASELINE LINEAR MODEL

In [ ]:
import datetime

baseline_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.LocationID),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_sum = pw.reducers.sum(pw.this.Occupancy),
        occ_count = pw.reducers.count(),
        cap = pw.reducers.max(pw.this.Capacity),
        location = pw.reducers.any(pw.this.LocationID)
    )
    .with_columns(
        occupancy_rate = (pw.this.occ_sum / pw.this.occ_count) / pw.this.cap,
        price = 10.0 + 5.0 * ((pw.this.occ_sum / pw.this.occ_count) / pw.this.cap)
    )
)


MODEL 2: DEMAND BASED PRICE FUNCTION

In [ ]:
import datetime
import pathway as pw

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.LocationID),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_sum = pw.reducers.sum(pw.this.Occupancy),
        occ_count = pw.reducers.count(),
        cap = pw.reducers.max(pw.this.Capacity),
        queue_sum = pw.reducers.sum(pw.this.QueueLength),
        traffic_high = pw.reducers.count(pw.this.TrafficConditionNearby == "high"),
        traffic_medium = pw.reducers.count(pw.this.TrafficConditionNearby == "medium"),
        traffic_low = pw.reducers.count(pw.this.TrafficConditionNearby == "low"),
        special_day_count = pw.reducers.sum(pw.this.IsSpecialDay),
        vehicle_car = pw.reducers.count(pw.this.VehicleType == "car"),
        vehicle_bike = pw.reducers.count(pw.this.VehicleType == "bike"),
        vehicle_truck = pw.reducers.count(pw.this.VehicleType == "truck"),
        location = pw.reducers.any(pw.this.LocationID),
    )
    # Step 1: Calculate demand_score
.with_columns(
    demand_score = (
        0.4 * ((pw.this.occ_sum / pw.this.occ_count) / pw.this.cap) +
        0.2 * (pw.this.queue_sum / pw.this.occ_count) +
        0.2 * (
            (1.5 * pw.this.traffic_high +
             1.0 * pw.this.traffic_medium +
             0.5 * pw.this.traffic_low) / pw.this.occ_count
        ) +
        0.1 * (pw.this.special_day_count / pw.this.occ_count) +
        0.1 * (
            (1.0 * pw.this.vehicle_car +
             0.5 * pw.this.vehicle_bike +
             1.5 * pw.this.vehicle_truck) / pw.this.occ_count
        )
    )
)

# Step 2: Normalize demand_score
    .with_columns(
    normalized_demand = pw.if_else(
        (pw.this.demand_score / 5.0) < 0.0, 0.0,
        pw.if_else(
            (pw.this.demand_score / 5.0) > 1.0, 1.0,
            pw.this.demand_score / 5.0
        )
    )
)


# Step 3: Compute final price
.with_columns(
    price = 10.0 + 10.0 * pw.this.normalized_demand
))


In [ ]:
df[['Timestamp', 'LocationID', 'Occupancy', 'Capacity',
    'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']].to_csv('parking_stream.csv', index=False)


In [ ]:
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig


In [ ]:
viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()


In [ ]:
delta_window.schema.columns()

In [ ]:
df = df.rename(columns={"LocationID": "SystemCodeNumber"})

# Step 1: Clean and save
df.columns = df.columns.str.strip().str.replace("'", "")
df.to_csv("lot_info.csv", index=False)

class LotSchema(pw.Schema):
    SystemCodeNumber: str
    Latitude: float
    Longitude: float

lot_info = pw.io.csv.read("lot_info.csv", schema=LotSchema)
# lot_info = pw.io.python.read(
#     df[["SystemCodeNumber", "Latitude", "Longitude"]].to_dict(orient="records"),
#     schema=LotSchema



In [ ]:
print(df.columns.tolist())


In [ ]:
# Step 1: Create lot_a and lot_b for self-join
lot_a = lot_info.select(
    SystemCodeA = pw.this.SystemCodeNumber,
    LatA = pw.this.Latitude,
    LonA = pw.this.Longitude
)

lot_b = lot_info.select(
    SystemCodeB = pw.this.SystemCodeNumber,
    LatB = pw.this.Latitude,
    LonB = pw.this.Longitude
)

# Perform join
lot_pairs = pw.join(lot_a, lot_b).filter(
    pw.left.SystemCodeA != pw.right.SystemCodeB
)


In [ ]:
lot_pairs_flat = lot_pairs.select(
    SystemCodeA = pw.left.SystemCodeA,
    LatA = pw.left.LatA,
    LonA = pw.left.LonA,
    SystemCodeB = pw.right.SystemCodeB,
    LatB = pw.right.LatB,
    LonB = pw.right.LonB
)

In [ ]:
df["distance"] = df.apply(lambda row: haversine(row["LatA"], row["LonA"], row["LatB"], row["LonB"]), axis=1)
